In [1]:
#default_exp rss_feeds 

In [2]:
#export
from operator import itemgetter

import numpy as np
import tqdm
import bs4
import feedparser
import seaborn as sns
import pandas as pd

In [3]:
%cd ..

/home/kuba/Projects/pytorch_hackathon


In [4]:
pd.set_option("display.max_colwidth", 250)

# RSS Feeds


In [5]:
#export
medium_publications = [
    'the-artificial-impostor',
    'pytorch',
    'dair.ai',
    'towards-artificial-intelligence',
    'swlh',
    '@ODSC',
    'doctrine',
    'paperswithcode'
]


medium_url_template = 'https://medium.com/feed/{}'
medium_url = medium_url_template.format(medium_publications[0])
medium_urls = [medium_url_template.format(publication) for publication in medium_publications]

In [6]:
#export
subreddits = [
    'MachineLearning',
    'deeplearning',
    'datascience',
    'cognitivelinguistics',
    'TopOfArxivSanity',
    'kaggle'
]

reddit_url_template = 'https://www.reddit.com/r/{}/.rss'
reddit_url = reddit_url_template.format(subreddits[0])
reddit_urls = [reddit_url_template.format(subreddit) for subreddit in subreddits]

In [7]:
#export


def get_article_text(article):
    article_html_content = article['content'][0]['value']
    article_text = bs4.BeautifulSoup(article_html_content).text
    return article_text


def get_feed_article_texts(feed):
    return [get_article_text(article) for article in feed['entries'] if 'content' in article.keys()]


def get_feed_article_df(feed):
    feed_df = pd.DataFrame.from_records(feed['entries'])
    feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)
    return feed_df


def add_field(df, field_name, values):
    df[field_name] = values
    return df

In [8]:
#export
paperswithcode_url = 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest' 
hackernews_url = 'https://news.ycombinator.com/rss' 
rss_feed_urls = [paperswithcode_url, hackernews_url] + medium_urls + reddit_urls

In [9]:
feed_urls = pd.read_table('data/feeds.txt', header=None).iloc[:,0].values

In [10]:
for url in rss_feed_urls:
    print(url)

https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest
https://news.ycombinator.com/rss
https://medium.com/feed/the-artificial-impostor
https://medium.com/feed/pytorch
https://medium.com/feed/dair.ai
https://medium.com/feed/towards-artificial-intelligence
https://medium.com/feed/swlh
https://medium.com/feed/@ODSC
https://medium.com/feed/doctrine
https://medium.com/feed/paperswithcode
https://www.reddit.com/r/MachineLearning/.rss
https://www.reddit.com/r/deeplearning/.rss
https://www.reddit.com/r/datascience/.rss
https://www.reddit.com/r/cognitivelinguistics/.rss
https://www.reddit.com/r/TopOfArxivSanity/.rss
https://www.reddit.com/r/kaggle/.rss


### Loading RSS feed articles to a dataframe

In [11]:
#export
def get_feed_df(feed_urls):
    feeds = [
        (feed_url, feedparser.parse(feed_url))
        for feed_url in tqdm.tqdm(feed_urls)
    ]
    return pd.concat(
        [
            add_field(get_feed_article_df(feed), 'feed', feed_url)
            for (feed_url, feed) in feeds
            if len(feed['entries']) > 0
        ]
    )

In [12]:
feed_df = get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:10<00:00,  1.59it/s]


In [13]:
feed_df.shape

(295, 22)

In [14]:
feed_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,published_parsed,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail
0,I2L-MeshNet: Image-to-Lixel Prediction Network for Accurate 3D Human Pose and Mesh Estimation from a Single RGB Image,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'I2L-MeshNet: Image-to-Lixel Prediction Network for Accurate 3D Human Pose and Mesh Estimation from a Single RGB Image'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/i2l-meshnet-image-to-lixel-prediction-network'}]",https://paperswithcode.com/paper/i2l-meshnet-image-to-lixel-prediction-network,"Most of the previous image-based 3D human pose and mesh estimation methods estimate parameters of the human mesh model from an input image. <p>Code: <a href=""https://github.com/mks0601/I2L-MeshNet_RELEASE"">https://github.com/mks0601/I2L-MeshNet_R...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Most of the previous image-based 3D human pose and mesh estimation methods estimate parameters of the human mesh model from an in...",https://paperswithcode.com/paper/i2l-meshnet-image-to-lixel-prediction-network,False,"[{'term': '3d hand pose estimation', 'scheme': None, 'label': None}, {'term': '3d human pose estimation', 'scheme': None, 'label': None}]",Most of the previous image-based 3D human pose and mesh estimation methods estimate parameters of the human mesh model from an input image. Code: https://github.com/mks0601/I2L-MeshNet_RELEASE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Polysemy Deciphering Network for Robust Human-Object Interaction Detection,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Polysemy Deciphering Network for Robust Human-Object Interaction Detection'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/polysemy-deciphering-network-for-robust-human'}]",https://paperswithcode.com/paper/polysemy-deciphering-network-for-robust-human,"To address this issue, in this paper, we propose a novel Polysemy Deciphering Network (PD-Net) that decodes the visual polysemy of verbs for HOI detection in three distinct ways. <p>Code: <a href=""https://github.com/MuchHair/PD-Net"">https://githu...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'To address this issue, in this paper, we propose a novel Polysemy Deciphering Network (PD-Net) that decodes the visual polysemy o...",https://paperswithcode.com/paper/polysemy-deciphering-network-for-robust-human,False,"[{'term': 'Human-object interaction detection', 'scheme': None, 'label': None}, {'term': 'Scene understanding', 'scheme': None, 'label': None}]","To address this issue, in this paper, we propose a novel Polysemy Deciphering Network (PD-Net) that decodes the visual polysemy of verbs for HOI detection in three distinct ways. Code: https://github.com/MuchHair/PD-Net",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cascade Graph Neural Networks for RGB-D Salient Object Detection,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Cascade Graph Neural Networks for RGB-D Salient Object Detection'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/cascade-graph-neural-networks-for-rgb-d'}]",https://paperswithcode.com/paper/cascade-graph-neural-networks-for-rgb-d,"Current works either simply distill prior knowledge from the corresponding depth map for handling the RGB-image or blindly fuse color and geometric information to generate the coarse depth-aware representations, hindering the performance of RGB-D...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-